<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_KoGPT_%EC%9A%94%EA%B8%B0%EC%9A%94_%EB%8B%B5%EB%B3%80%EC%83%9D%EC%84%B1_(%EC%A7%84%ED%96%89%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cpu'

# 데이터 정제

In [117]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


## 정규표현식

In [118]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [119]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-119-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [120]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-120-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


## 1) 고객id 처리
## - ~님 -> 고객님

In [121]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


<ipython-input-121-ef52f5ba6171>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


In [122]:
customer_sentence_id = []

for review in reply_list:
        if '님' in review :
          customer_sentence_id.append(review)

In [123]:
customer_sentence_id

['이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다 항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다 ',
 '좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로가 싹~ 사라지네요! 주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요 오늘도 화이팅 있게 마무리하시길 바라요 ',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요? 저희 메뉴 칭찬해 주셔서 감사합니다 고객님! 늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요~항상 노력하고 발전하도록 하겠습니다 항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다 ',
 '저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다! 언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까! 매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다 하시는 일 모두 번창하시길 진심으로 바라고 있을게요 ',
 '정성 가득한 리뷰 작성해주셔서 감사합니다 맛있는 시간이 되셨다니 정말 다행입니다 저희 매장은 재료부터 맛, 포장, 서비스 모든 면에 진심을 담아 고객님이 완벽하게 행복하실 수 있도록 노력하겠습니다 언제든 신호만 주시면 바로 달려가는 매장이 되겠습니다! 다음에 또 만나요! ',
 '즐거운 순간을 저희와 함께해주셔서 감사합니다 믿고 찾아 주셔서 진심으로 감사드립니다 고객님의 신뢰에 항상 보답하는 매장이 되기 위해서 끊임없이 노력하겠습니다! 저희 매장 다시 와주실꺼죠? 언제든 신호만 주시면 바로 달려가도록 하겠습니다! 다음에 또 만나요! ',
 '귀한 발걸음 감사합니다! 고객님의 리뷰는 저희를 성장시키는 원동력이 됩

## 2) 가게명 처리
- reply에 가게명이 들어가있는 경우에 store 칼럼에 가게명 기재, 모델이 함께 학습할 수 있도록 처리
- '저희 가게'로 처리?



In [124]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [125]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

In [126]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '대찌',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕',
 '깨돌이김밥',
 '울진죽변항',
 'GTS버거',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '초밥대통령',
 '올데이케밥',
 '

In [15]:
len(store_name)

125

In [127]:
store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

In [128]:
store_etc_name # 지점명, 메뉴명 포함

['방이점',
 '로스터스',
 '가락점',
 '헬리오시티점',
 '석촌점',
 '송파점',
 '마라샹궈',
 '송파점',
 '가락점',
 '송파점',
 '삼전점',
 '송파점',
 '헬리오시티점',
 '김치찌개',
 '송파점',
 '송파점',
 '본점',
 '연어',
 '미아본점',
 '본점',
 '성신여대역점',
 '대학로점',
 '미아본점',
 '성북점',
 '중국',
 '마라탕',
 '덮밥',
 '요리',
 '전문점',
 '성북점',
 '동대문구점',
 '성북점',
 '성북점',
 '성북점',
 '북서울꿈의숲점',
 '닭발',
 '김치찜',
 '혜화막내딸지점',
 '마약볶음밥',
 '대파생고기김치찌개',
 '정육왕국물두루치기',
 '김치찌개',
 '부천점',
 '부천점',
 '부평점',
 '부천부평점',
 '부천점',
 '본가진죽',
 '본점',
 '분식',
 '부평점',
 '부천점',
 '신중동점',
 '부천점',
 '부천시청점',
 '김치찌개',
 '도촌점',
 '야탑점',
 '판교점',
 '야탑점',
 '수제돈까스',
 '파스타',
 '정자본점',
 '미금본점',
 '전기구이통닭',
 '미금점',
 '분당정자점',
 '분당미금역점',
 '죽전점',
 '용인수지점',
 '샐러드',
 '카마쿠라',
 '서현본점',
 '분당점',
 '카페',
 '김치찌개',
 '감자탕']

In [129]:
short_name = []
for name in store_name:
    for length in [2, 3, 4, 5, 6, 7]:
        cut = name[:length]
        short_name.append(cut)
    # if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
    #     cut = name[:6]
    #     short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
print(len(short_name), short_name)

466 ['초밥대통령', '부산오빠', '우리집반찬도', '병천토속', '최가', '청춘', '왕빈자삼파', '송파만다린', '밈피', '닭강점', '송파만다', '스시사소우', '삽교', '우시장국밥', '고씨네카', '손수', '라화방마', '야키토', '큰아들백순', '우시장', '과일', '창타이누', '행복', '모모타코', '나인곱창', '짜글이가', '청년치킨', '조형훈족발보쌈', '수상한삼', '피자보이', '부산오빠1인', '시민보쌈족', '공주', '피제이', '천년닭강', '수피자', '큰아들', '최고남제육', '부대찌개자', '행복한찜', '승도리네곱도리', '왕빈', '청춘식당묵은지', '토핑폭탄', '랜돌프', '스시사소', '황궁쟁반짜', '코브', '신자', '손수경의육회', '홍스족발', '수상', '직화삼겹직구삼', '짜글이가돼었', '부산', '워커', '초밥대', '천년닭', '모어', '승도리네', '직화삼겹직구', '올데이케', '직화삼', '잭아저', '승도리', '짜글이', '행복한', '부대찌', '피자보이시나', '대한냉', '올데이', '쇼부덮', '이끌림마라', '쇼부덮밥', '깨돌이김밥', '호랑이', '시민보', '청춘식당', '과일에반하다', '꼬알라파', '플러스', '승도리네곱도', '이끌림마라탕', '더팔당매운', '신자매김치', '여왕의브', '춘리', '울진죽변항', '땅끝수', '병천토속순대', '홍대개미', '청년치', '폴트버거', '걸작떡볶', '고기혁', '대한', '갈비', '저팔', '카산', '나인', '구구족', '쇼부', '부성초밥', '구구', '마라의신마라탕', '라화방마라탕', '부산오빠1인국', '고씨', '수제죽', '고기듬', '꼬알라파이', '최고남', '더팔당매', '코브라독', '춘리마라탕', '우리집반', '한식세끼', '모모타코야키', '뜸들이', '닭장수섭', '짚신스', '무국', '여왕의브런치', '울진', '우시', '여왕의', '1988응답', '

- 형용사, 부사, 등등인 경우는 제외하기 / 등등의 방법 필요함




In [130]:
# short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
drop_name = ['OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',]
# drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

# 명사인 경우에만 남기기 형용사, 부사 등이


short_name = [col for col in short_name if col not in drop_name]
print(len(short_name),short_name)

423 ['초밥대통령', '부산오빠', '우리집반찬도', '병천토속', '최가', '왕빈자삼파', '송파만다린', '밈피', '닭강점', '송파만다', '스시사소우', '우시장국밥', '고씨네카', '라화방마', '야키토', '큰아들백순', '우시장', '창타이누', '모모타코', '나인곱창', '짜글이가', '청년치킨', '조형훈족발보쌈', '수상한삼', '피자보이', '부산오빠1인', '시민보쌈족', '피제이', '천년닭강', '수피자', '최고남제육', '부대찌개자', '행복한찜', '승도리네곱도리', '왕빈', '청춘식당묵은지', '토핑폭탄', '랜돌프', '스시사소', '황궁쟁반짜', '코브', '신자', '손수경의육회', '홍스족발', '직화삼겹직구삼', '짜글이가돼었', '워커', '초밥대', '천년닭', '모어', '승도리네', '직화삼겹직구', '올데이케', '직화삼', '잭아저', '승도리', '짜글이', '부대찌', '피자보이시나', '대한냉', '올데이', '쇼부덮', '이끌림마라', '쇼부덮밥', '깨돌이김밥', '호랑이', '시민보', '청춘식당', '과일에반하다', '꼬알라파', '플러스', '승도리네곱도', '이끌림마라탕', '더팔당매운', '신자매김치', '여왕의브', '춘리', '울진죽변항', '땅끝수', '병천토속순대', '홍대개미', '청년치', '폴트버거', '걸작떡볶', '고기혁', '갈비', '저팔', '카산', '나인', '구구족', '부성초밥', '구구', '마라의신마라탕', '라화방마라탕', '부산오빠1인국', '고씨', '수제죽', '고기듬', '꼬알라파이', '최고남', '더팔당매', '코브라독', '춘리마라탕', '우리집반', '한식세끼', '모모타코야키', '뜸들이', '닭장수섭', '짚신스', '무국', '여왕의브런치', '우시', '여왕의', '1988응답', '라화방', '스시', '천년닭강정', '비오키', '윤희횟', '구래상회', '탐나는피자', '신자매김치찜', '삽교원조두리', '뜸들', '승

In [20]:
# short_etc_name = []
# for name in store_etc_name:
#     for length in [2, 3, 4, 5, 6, 7]:
#         cut = name[:length]
#         short_etc_name.append(cut)
#     if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#         cut = name[:6]
#         short_etc_name.append(cut)

# short_etc_name = [name for name in short_etc_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_etc_name = list(set(short_etc_name))
# print(len(short_etc_name), short_etc_name)

In [21]:
# train['store'] = ""
# for name in short_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=short_name) # 각 가게명 컬럼 삭제

# train.head()

<ipython-input-21-1e46b9def5a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['store'] = ""
<ipython-input-21-1e46b9def5a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
<ipython-input-21-1e46b9def5a1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,review,reply,store
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,


In [131]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '대찌',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕',
 '깨돌이김밥',
 '울진죽변항',
 'GTS버거',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '초밥대통령',
 '올데이케밥',
 '

- 가게명을 특정 태그로 변환하기

In [135]:
def replace_store_names(text, store_name):
    for store in store_name:
        text = text.replace(store, "<store>")
    return text

In [136]:
for store in store_name:
    for i, review in enumerate(train['reply']):
        if store in review:
            train['reply'][i] = replace_store_names(review, [store])

In [137]:
changed_rows = train[train['reply'].str.contains("<store>")]
changed_rows

,review,reply
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...
43,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,<store>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 때 배달 시...
47,"메인은 좋았는데, 아. 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 이상해...",안녕하세요 <store> 입니다. 우선 감사하게도 저희 매장을 찾아 주셨는데 불편함...
51,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,<store>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 때 배달 시...
...,...,...
18502,"여기 미역국이 맛있어서 주문했네요~ 죽과 미역국, 어르신께서 맛있게 드셨습니다 감사...",안녕하세요 어르신이 맛있게 드셨다니 정말 기쁘네요 멋진 칭찬 남겨주셔서 정말 감사드...
18507,오늘 생일인데 더 잘먹었습니다,안녕하세요 다시한번 생일축하드립니다 소중한 생일식사에 <store> 수영점을 찾아주...
18508,"부모님 보내드렸는데, 양도 넉넉하고 따뜻하게 배달되어 맛있게 드셨다네요. 감사합니다",안녕하세요 귀한 식사에 <store> 수영점을 찾아주셔서 정말 감사드립니다 고객님께...
18522,믿고먹는밥 찐맛탱 굿,찐맛탱집 <store>입니다 맛있게 드셔주셔서 항상 감사드립니다! 신메뉴3종 출시 ...


In [138]:
for store in store_name:
    for i, review in enumerate(train['review']):
        if store in review:
            train['review'][i] = replace_store_names(review, [store])

<ipython-input-138-72814c8beb6c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'][i] = replace_store_names(review, [store])


In [139]:
changed_rows = train[train['review'].str.contains("<store>")]
changed_rows

,review,reply
361,<store>에서 로제는 처음먹어봣는데 진짜 맛있네요,로제가 인기가 많아요~ 로제는 크림 때문에 맵기가 중화가 되버리니 한단계 더 맵게 ...
381,<store>는 고구마 무스 안한게 더 맛있네요! 배달이 너무 오래 걸린것만 빼면 ...,저도 고구마무스 안한게 훨씬 맛있습니다. 다 제 입맛은 아니듯 고구마무스 드시는 분...
396,<store> 예전 동네에서 못 찾았다가 이사 와서 시켰는데 똑같이 맛있네요 치즈 ...,저는 음식장사 야박하면 망한다 생각하는 사람으로서 최대한 넉넉하게 드리려고 합니다....
397,<store>는 처음먹어봤는데 다른곳보다 오뎅도 많고 양도 괜찮았어요. 잘먹었습니다!,"팍팍드리고 있어요~이번에 오뎅도 프리미엄으로 추가되었구요, 올만에 들려. 주시면 더..."
414,<store>는 첨 시켜보는 곳인데 너무 좋아요 개인적인 요청사항이 있었어서 전화로...,떡볶이는 콜라죠!이렇게 저와 맘이 통하는 댓글을 보니 마음이 기쁘네요 앞으로도 비밀...
...,...,...
18618,제가 먹어본 <store> 중에 양 제일 많았어요! 배달도 빨리 오고 양파 조금 더...,극찬 너무 감사드립니다 맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 ...
18636,스테이크 굽기 좋아서 너무 맛있게 먹었습니다. 고기 추가가 없어 아쉬웠지만. 덕분에...,고객님 요청으로 드디어 요기요에도 고기추가 추가 했습니다! 배.민에 이어 요기요 까...
18641,믿고먹는 <store> 짱짱맛있어요 넘 맛있어서 좋아요,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다
18678,<store> 넘 마싯어요 부산 놀러가서 첫끼가 <store>였는대 참 순삭했습니다,헉 부산에서 처음.! 감사합니당


In [140]:
store_mask = []

for review in train['reply']:
  if '<store>' in review:
      print(review)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 배달관련해서 저희가 할 수 있는 일이 없어서 너무 안타깝습니다 그래도 맛에 만족하셨다니 정말 다행입니다 ! 주문해주셔서 감사합니다 ! 언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다 변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다 족발보쌈 생각나는날은 언제나 <store>과 함께 해주세요 다음 주문도 만족스러운 음식을 제공해드릴게요
항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다 변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다 족발보쌈 생각나는날은 언제나 <store>과 함께 해주세요 다음 주문도 만족스러운 음식을 제공해드릴게요
항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다 변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다 족발보쌈 생각나는날은 언제나 <store>과 함께 해주세요 다음 주문도 만족스러운 음식을 제공해드릴게요
항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다 변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다 족발보쌈 생각나는날은 언제나 <store>과 함께 해주세요 다음 주문도 만족스러운 음식을 제공해드릴게요
항상 웃음이 가득한 <store>을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각하며 조리하고 있습니다 변함없는 모습으로 정성 가득 준비한 음식을 고객님께 제공해 행복한 시간을 만들어드리겠습니다 족발보쌈 생각나는날은 언제나 <store>과 함께 해주세요 다음 주문도 만족스러운 음식을 제공해드릴게요
항상 웃음이 가득한 <store>을 찾아주셔서 감사합

In [141]:
# train[train['reply'].str.contains(short_name[2])].index

In [142]:
# train['store'][10216]

In [25]:
# # 매장 이름별 컬럼 생성(안 그러면 반복문 돌릴 때 덮어쓰기가 됨)
# for store in store_name:
#     train[store] = train['reply'].apply(lambda x: store if store in x else '')

# # 하나의 컬럼으로 합치고 필요 없는 컬럼 삭제
# train['store'] = train[store_name].apply(lambda x: ''.join(x[x.notna()]), axis=1)
# train = train.drop(columns=store_name) #
# train.head()

In [ ]:
# mask = train['store'] != ''
# train[mask]    #약 1/3 6330개 리뷰답변에 가게명 확인 / 글자 2~7 추가 -> 11379 리뷰

In [27]:
# train[mask]['store']

16       1988응답,1988응,1988응답하
23       1988응답,1988응,1988응답하
29                     플러스,플러
43       1988응답,1988응,1988응답하
47       1988응답,1988응,1988응답하
                 ...         
18522                홍대개미,홍대개
18532                      고기
18537                홍대개미,홍대개
18616                      고기
18636                      고기
Name: store, Length: 9791, dtype: object

- store 부분을 <store> 로 처리하기

## 3) 지점명 처리
- 전국일반음식점 표준 데이터 : https://www.data.go.kr/data/15096283/standard.do
- 행정구역 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 region 칼럼에 추가 or 삭제
- 지점명의 경우 보통 가게명과 함께 등장하기 때문에, 빈칸으로 삭제해도 무방

- 지점명을 특정 태그로 변경

In [144]:
# restaurant_seoul = pd.read_csv(f"{DATA_PATH}소상공인시장진흥공단_상가(상권)정보_서울_202212.csv")

In [145]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

<ipython-input-145-afda4487c689>:1: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')


In [146]:
restaurant_list['소재지전체주소']

0                   서울특별시 동대문구 장안동 382-1
1               서울특별시 마포구 망원동 386-2번지 지층
2                 서울특별시 마포구 서교동 403-14번지
3              서울특별시 마포구 서교동 363-5번지 B1동
4          경기도 안양시 동안구 호계동 930-30번지 지상1층
                       ...              
2086169             광주광역시 서구 용두동 306번지 1
2086170              광주광역시 남구 양림동 374 2층
2086171     광주광역시 남구 월산동 1048-182번지 (1층)
2086172        광주광역시 서구 화정동 332-9번지 (1층)
2086173          광주광역시 서구 화정동 70-61 (1층)
Name: 소재지전체주소, Length: 2086174, dtype: object

In [ ]:
store_etc_name

In [54]:
def replace_store_names(text, region_name):
    for region in region_name:
        text = text.replace(region, "<region>")
    return text

In [ ]:
for region in region_name:
    for i, review in enumerate(train['reply']):
        if region in review:
            train['reply'][i] = replace_store_names(review, [region])

In [ ]:
changed_rows = train[train['reply'].str.contains("<store>")]
changed_rows

,review,reply,store
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...,"1988응답,1988응,1988응답하"
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...,"1988응답,1988응,1988응답하"
43,맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간...,<store>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 때 배달 시...,"1988응답,1988응,1988응답하"
47,"메인은 좋았는데, 아. 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 이상해...",안녕하세요 <store> 입니다. 우선 감사하게도 저희 매장을 찾아 주셨는데 불편함...,"1988응답,1988응,1988응답하"
51,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,<store>을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 때 배달 시...,"1988응답,1988응,1988응답하"
...,...,...,...
18502,"여기 미역국이 맛있어서 주문했네요~ 죽과 미역국, 어르신께서 맛있게 드셨습니다 감사...",안녕하세요 어르신이 맛있게 드셨다니 정말 기쁘네요 멋진 칭찬 남겨주셔서 정말 감사드...,"보돌,보돌미,보돌미역"
18507,오늘 생일인데 더 잘먹었습니다,안녕하세요 다시한번 생일축하드립니다 소중한 생일식사에 <store> 수영점을 찾아주...,"보돌,보돌미,보돌미역"
18508,"부모님 보내드렸는데, 양도 넉넉하고 따뜻하게 배달되어 맛있게 드셨다네요. 감사합니다",안녕하세요 귀한 식사에 <store> 수영점을 찾아주셔서 정말 감사드립니다 고객님께...,"보돌,보돌미,보돌미역"
18522,믿고먹는밥 찐맛탱 굿,찐맛탱집 <store>입니다 맛있게 드셔주셔서 항상 감사드립니다! 신메뉴3종 출시 ...,"홍대개미,홍대개"


## 4) 메뉴명 처리
-
- 일반 메뉴 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 menu 칼럼에 추가 or 삭제
- 특수 메뉴 리스트 -> ex. 저희 ~~커피, 저희 ~~찜 등

In [85]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [86]:
food_list = set(food_list['대표식품명'].tolist())

In [104]:
food_list = list(food_list)

In [103]:
ygy_menu_list = ['도시락','카레','커피','국밥','마라탕','마라샹궈','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭']

In [105]:
menu_name = food_list + ygy_menu_list

In [106]:
len(menu_name)

805

In [107]:
short_name = []
for name in menu_name:
    for length in [2, 3, 4, 5, 6, 7]:
        cut = name[:length]
        short_name.append(cut)
    # if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
    #     cut = name[:6]
    #     short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
print(len(short_name), short_name)

1823 ['풋고추된장', '꽁치', '돼지갈비구이', '무나물무', '단호박', '고구마맛', '보리밥', '호박전', '샤브샤브', '시래기나', '달걀탕', '감자빵', '소갈', '소보로빵', '호박오가리나', '잡곡밥', '동태찜', '오므라이', '미역', '동파육', '도토', '찰시', '돼지갈비', '멸치조', '잔멸치볶음', '참치회덮밥', '병어조', '수제비', '꽃게콩나물찜', '샤베', '부추', '북어강', '떡갈비', '햄채소', '소고기육', '참치', '어묵볶음', '멸치조림', '상추', '오징어산적', '차조밥', '마늘장아찌무', '순두부찌', '고들빼기김', '알탕', '게살', '연포', '오징어국', '라조기', '무조림', '건새우볶', '민들레', '돼지고', '오리고기구이', '메밀국', '소고기전골', '재첩', '시리', '들깨', '달걀', '부대찌', '모자반', '풋고추', '마늘장아', '가오리콩', '미소된장국', '치즈빵', '가래떡', '꼬치구이', '더덕장', '배추', '돼지불고기', '닭고기육개장', '닭볶음', '웨이', '치아바타', '대창', '런천미트', '토스트', '미역초무', '츄러스', '오징어젓', '초코', '수수밥', '넙치(광', '오징어산', '냉이', '밀크티', '잡채밥', '쌍화차', '소세지빵', '어묵볶', '분짜', '꼬리', '무말랭이', '병어', '풋고추찜', '초당순두부', '기타음료', '마라탕', '꽃게장', '견과', '증편', '영양', '다시마무침', '무국', '캘리포니아롤', '도라지생채', '닭구', '곱창전골', '열무김', '알밥', '가죽나물무', '새송이버섯전', '오리불', '녹두빈대', '닭튀김(', '해초무', '초당', '애호박찜', '가오리콩나', '밀푀유나', '장어탕', '쌀죽(흰죽', '오징어채무', '계란', '굴무', '황태', '베이글', '수수부', '씀바귀나', '참치회', '닭발구', '도라지나물무', '

In [108]:
# short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
drop_name = ['비스','다시','번']
# drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

# 명사인 경우에만 남기기 형용사, 부사 등이


short_name = [col for col in short_name if col not in drop_name]
print(len(short_name),short_name)

1820 ['풋고추된장', '꽁치', '돼지갈비구이', '무나물무', '단호박', '고구마맛', '보리밥', '호박전', '샤브샤브', '시래기나', '달걀탕', '감자빵', '소갈', '소보로빵', '호박오가리나', '잡곡밥', '동태찜', '오므라이', '미역', '동파육', '도토', '찰시', '돼지갈비', '멸치조', '잔멸치볶음', '참치회덮밥', '병어조', '수제비', '꽃게콩나물찜', '샤베', '부추', '북어강', '떡갈비', '햄채소', '소고기육', '참치', '어묵볶음', '멸치조림', '상추', '오징어산적', '차조밥', '마늘장아찌무', '순두부찌', '고들빼기김', '알탕', '게살', '연포', '오징어국', '라조기', '무조림', '건새우볶', '민들레', '돼지고', '오리고기구이', '메밀국', '소고기전골', '재첩', '시리', '들깨', '달걀', '부대찌', '모자반', '풋고추', '마늘장아', '가오리콩', '미소된장국', '치즈빵', '가래떡', '꼬치구이', '더덕장', '배추', '돼지불고기', '닭고기육개장', '닭볶음', '웨이', '치아바타', '대창', '런천미트', '토스트', '미역초무', '츄러스', '오징어젓', '초코', '수수밥', '넙치(광', '오징어산', '냉이', '밀크티', '잡채밥', '쌍화차', '소세지빵', '어묵볶', '분짜', '꼬리', '무말랭이', '병어', '풋고추찜', '초당순두부', '기타음료', '마라탕', '꽃게장', '견과', '증편', '영양', '다시마무침', '무국', '캘리포니아롤', '도라지생채', '닭구', '곱창전골', '열무김', '알밥', '가죽나물무', '새송이버섯전', '오리불', '녹두빈대', '닭튀김(', '해초무', '초당', '애호박찜', '가오리콩나', '밀푀유나', '장어탕', '쌀죽(흰죽', '오징어채무', '계란', '굴무', '황태', '베이글', '수수부', '씀바귀나', '참치회', '닭발구', '도라지나물무', '

In [92]:
# train['menu'] = ""
# for name in tqdm(short_name):
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['menu'] = train.apply(lambda row: row['menu'] + ',' + row[name] if row[name] else row['menu'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['menu'] = train['menu'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=short_name) # 각 가게명 컬럼 삭제

# train.head()

  0%|          | 0/1824 [00:00<?, ?it/s]

,review,reply,store,menu
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",,비스
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,다시
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,"번,다시,소"
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,,


1820 ['풋고추된장', '꽁치', '돼지갈비구이', '무나물무', '단호박', '고구마맛', '보리밥', '호박전', '샤브샤브', '시래기나', '달걀탕', '감자빵', '소갈', '소보로빵', '호박오가리나', '잡곡밥', '동태찜', '오므라이', '미역', '동파육', '도토', '찰시', '돼지갈비', '멸치조', '잔멸치볶음', '참치회덮밥', '병어조', '수제비', '꽃게콩나물찜', '샤베', '부추', '북어강', '떡갈비', '햄채소', '소고기육', '참치', '어묵볶음', '멸치조림', '상추', '오징어산적', '차조밥', '마늘장아찌무', '순두부찌', '고들빼기김', '알탕', '게살', '연포', '오징어국', '라조기', '무조림', '건새우볶', '민들레', '돼지고', '오리고기구이', '메밀국', '소고기전골', '재첩', '시리', '들깨', '달걀', '부대찌', '모자반', '풋고추', '마늘장아', '가오리콩', '미소된장국', '치즈빵', '가래떡', '꼬치구이', '더덕장', '배추', '돼지불고기', '닭고기육개장', '닭볶음', '웨이', '치아바타', '대창', '런천미트', '토스트', '미역초무', '츄러스', '오징어젓', '초코', '수수밥', '넙치(광', '오징어산', '냉이', '밀크티', '잡채밥', '쌍화차', '소세지빵', '어묵볶', '분짜', '꼬리', '무말랭이', '병어', '풋고추찜', '초당순두부', '기타음료', '마라탕', '꽃게장', '견과', '증편', '영양', '다시마무침', '무국', '캘리포니아롤', '도라지생채', '닭구', '곱창전골', '열무김', '알밥', '가죽나물무', '새송이버섯전', '오리불', '녹두빈대', '닭튀김(', '해초무', '초당', '애호박찜', '가오리콩나', '밀푀유나', '장어탕', '쌀죽(흰죽', '오징어채무', '계란', '굴무', '황태', '베이글', '수수부', '씀바귀나', '참치회', '닭발구', '도라지나물무', '

In [109]:
train

,review,reply,store,menu
0,사진보다 실물이 백배 맛있어요~ <menu> 양도 많아요~ 배달시간도 딱 맞춰서 보...,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",,비스
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,다시
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,<menu>중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주...,,"번,다시,소"
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,,
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,,
18682,요청사항을 항상 잘 들어주셨는데 <store> 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,,
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,,
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,,


In [150]:
menu_name = set(menu_name)
menu_name = list(menu_name)

In [147]:
# mask = train['menu'] != ''
# train[mask]

KeyError: ignored

In [153]:
def replace_menu_names(text, menu_name):
    for menu in menu_name:
        text = text.replace(menu, "<menu>")
    return text

In [154]:
for menu in menu_name:
    for i, review in enumerate(train['reply']):
        if menu in review:
            train['reply'][i] = replace_menu_names(review, [menu])

<ipython-input-154-742266b9bfe1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = replace_menu_names(review, [menu])


In [155]:
changed_rows = train[train['reply'].str.contains("<menu>")]
changed_rows

,review,reply
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 고객님들의 배고픔을 저희는 지...
16,저저<menu>달에 맛있게 먹어서 또 시켰는데 <menu> 양이 엄청 줄었네요 .<...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대...,안녕하세요 <store> 입니다. 남겨 주신 리뷰 잘 읽어보았습니다. 먼저 매장 상...
29,양도 많고 진짜 맛있어요! 시원한 묵사발로 더위도 가라앉히고 밥이랑 <menu> 양...,"저희 매장을 선택해주시고, 리뷰 남겨주셔서 감사합니다! 메뉴를 드시다가 양이 모자<..."
...,...,...
18615,늘 잘 먹고 있습니다~,"신메뉴 미니 <menu> 시리즈, 매운<menu><menu> 출시 새해 복 많이 ..."
18616,영쉬 믿고 먹는 <store> <menu>입니당! 요청 드린대로 숙주랑 양파 더 주...,"신메뉴 매운 <menu> <menu> , 미니 <menu> <menu> 출시 맛있..."
18620,<menu><menu>행사너무좋아요,<menu> <menu>입니다 헤헷 맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! ...
18636,<menu> 굽기 좋아서 너무 맛있게 먹었습니다. <menu> 추가가 없어 아쉬웠지...,고객님 요청으로 드디어 요기요에도 <menu>추가 추가 했습니다! 배.민에 이어 요...


In [156]:
for menu in menu_name:
    for i, review in enumerate(train['review']):
        if menu in review:
            train['review'][i] = replace_menu_names(review, [menu])

In [158]:
changed_rows = train[train['review'].str.contains("<menu>")]
changed_rows

,review,reply
0,사진보다 실물이 백배 맛있어요~ <menu> 양도 많아요~ 배달시간도 딱 맞춰서 보...,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 <menu>도 맛있고 묵사발도 맛있...,귀한 발걸음 감사합니다! 고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~ 내 가...
16,저저<menu>달에 맛있게 먹어서 또 시켰는데 <menu> 양이 엄청 줄었네요 .<...,저희 <store>의 메뉴를 주문해 주셔서 감사합니다. 양이 줄었다고 느끼셨군요. ...
17,진짜 너무 자주 시켜먹어욧 소스가 삸싹 시큼한게 넘나 맛있고 <menu>도두툼하고 ...,안녕하세요! 저희 매장을 이용해 주셔서 진심으로 감사드려요! 저희의 메뉴가 만족스러...
18,<menu>가 좀 매워요 맛있게 잘먹었습니다,"저희 매장을 선택해주시고, 리뷰 남겨주셔서 감사합니다 단순히 메뉴가 아니라 고객님의..."
...,...,...
18656,<menu> 매운소스 따로 부탁드렸는데 요청대로 해주셔서 맛있게 잘 먹었습니다~큐브...,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다
18658,<menu>도 넉넉히주시고 친절하세요 맛있습니다,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다
18659,맛있어요 <menu>도넉넉히주셨어요,맛있게 드셔주셔서 감사합니당 리뷰 감사합니다! 다음에 또 뵙겠습니다
18679,맛있어요 <menu> 양 짱 많아요,감사합니다! 다음에도 이용해주세용


In [159]:
train.replace(['<menu><menu>','<menu> <menu>'], '<menu>')

,review,reply
0,사진보다 실물이 백배 맛있어요~ <menu> 양도 많아요~ 배달시간도 딱 맞춰서 보...,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...
...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 <store> 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용


## 5) 시간, 장소, 매장번호, 슬로건 등 고유 정보 처리
- 48시간, 24시간,02, 031, 번호 등

In [ ]:
for reply in train['reply'] :


## 맞춤법

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

fatal: destination path 'py-hanspell' already exists and is not an empty directory.


In [ ]:
!cd py-hanspell

In [ ]:
!pip install py-hanspell/

Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=185279daad2ad8efc194e88ab8b4be9737b2aeab0fb2850753abc80bc5fc66d7
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell
  Attempting uninstall: py-hanspell
    Found existing installation: py-hanspell 1.1
    Uninstalling py-hanspell-1.1:
      Successfully uninstalled py-hanspell-1.1


In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
correct_spell(train['review'][0])

'사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱 맞춰서 보내주셨어요~~~~'

In [ ]:
train.head()

,review,reply,store
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두...",
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...,


In [ ]:
correct_spell('감사합니다!!! 다음에도 이용해주세용')

'감사합니다!!! 다음에도 이용해 주세요'

In [ ]:
train['review'][:10].apply(correct_spell)

0    사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...
1                     리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
2                   잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                    굿 말이 필요 없는 메뉴였습니다
4                                        맛있어요 양도 많고 좋음
5                                           양 많고 맛있었어요
6    처음 주문했는데 맛있어서 다 먹었어요   리뷰 묵사발도 꼭 드세요! 맛있게 잘 먹었...
7                           가성비 아주 만족합니다. 맛있게 잘 먹었습니다.
8                                          맛있게 먹었습니다!!
9    새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다...
Name: review, dtype: object

In [ ]:
train['reply'][:10].apply(correct_spell)

0    이렇게 저희 매장 찾아주셔서 감사드립니다!!  맛뿐만 아니라 양도, 서비스도 모두 ...
1    좋은 리뷰 덕분에 기분이 행복해지는 것 같아요   고객님께서 맛있다고 말씀 주시니 ...
2    소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.   저희를 찾아 주신 만...
3    수많은 매장들을 헤치고 와주셔서 감사합니다!!  저희 매장의 메뉴를 제대로 즐기신 ...
4    수많은 매장들을 헤치고 와주셔서 감사합니다   메뉴 그 자체로 행복을 드리고 싶었는...
5    저희 매장 찾아주신 것도 감사한데 리뷰까지.. 감동입니다     고객님들의 배고픔을...
6    정성 가득한 리뷰 작성해 주셔서 감사합니다   맛있는 시간이 되셨다니 정말 다행입니...
7    즐거운 순간을 저희와 함께해 주셔서 감사합니다   믿고 찾아 주셔서 진심으로 감사드...
8    귀한 발걸음 감사합니다!!  아낌없이 칭찬해 주셔서 기분이 좋습니다   저희 매장은...
9    귀한 발걸음 감사합니다!!  고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~~ ...
Name: reply, dtype: object

In [ ]:
# tqdm.pandas().  # 오래걸려서 우선 모델 학습시에 맞춤법 처리 된걸 넣을 필요는 없을 것 같고, 답변을 내뱉을때 맞춤법 처리 기능을 함수에 추가하기로

# train['review'] = train['review'].progress_apply(correct_spell)
# train['reply'] = train['reply'].progress_apply(correct_spell)

In [ ]:
# train.to_csv(f'{DATA_PATH}train_posneg_grammarchecked.csv',index=False)

# 데이터 분석/증강/변형

## 별점 비율 확인

In [ ]:
# train = pd.read_csv(f"{DATA_PATH}train_preprocessed.csv")

In [ ]:
train_rate = pd.concat([train, yogiyo['별점']],axis=1)
train_rate

,review,reply,store,별점
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두...",,5
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,,5
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...,,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...,,5
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용,,5
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용,,4
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용,,5
18684,잘 먹었어요~~~~~,감사합니다!!! 다음에도 이용해주세용,,5


In [ ]:
for i in range(1, 6):
    num_rating = train_rate[train_rate['별점'] == i].shape[0]
    globals()[f'rate_{i}'] = train_rate[train_rate['별점'] == i]
    print(f"별점 {i}: {num_rating}, {round(num_rating / train_rate.shape[0] * 100)}%")

별점 1: 92, 0%
별점 2: 64, 0%
별점 3: 179, 1%
별점 4: 560, 3%
별점 5: 17791, 95%


In [ ]:
92+64+179

335

In [ ]:
rate_1

,review,reply,store,별점
96,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,안녕하세요 1988응답하라추억의옛날도시락 입니다. 주신 리뷰 잘 읽었습니다. 뒤돌...,1988응답하라추억의옛날도시락,1
144,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다. 저희 1988응답하라추억의옛날도시락은 언제...,1988응답하라추억의옛날도시락,1
252,만들어 놓은지 한참 된 커피 .배달 예정 시간 25분이 지났어요 .,안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 고객님들이 상황이 여유치가...,,1
363,밥이 쉬었어요... 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가...,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요. 진짜 먼 정신이 이상한 사람인가 하루에...,,1
401,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주문어플에 렉이 자주 있어서.. 여러가지로 혼돈이 있어 정말 죄송하게 되었...,,1
...,...,...,...,...
16302,여기 원래 세트메뉴에 원래 탕수육 소스 안나오나요?? 맛있어서 가끔 시켜멌었는데 세...,죄송합니다 비네리는 주말이다보니 배달이많이밀려 실수가있었던모양입니다 다시한번사과의말...,,1
16401,하진짜 사진도 아깝네요 면이 젓가락으로 집으면 그냥으깨져서 먹을수가없네요 콜라는...,안녕하세요 우선 사과말씀부터드립니다 정말죄송합니다 오늘비가많이오다보니 고객님들이 ...,,1
17313,사장님 처럼 사장님만보기 기능있면좋겠지만만족하지만개선되었으면할때쓰고아니면안시키고말...,저희 윤희횟집을 찾아주셔서 감사드립니다 고객님께서 저희에게 충고해주신것에대해 감사드...,윤희횟집,1
17679,음료 맛도 별로였고 빨대도 안주고 마감전이라 그런지 샌드위치 시켰는데 먹다가 그냥...,고객님 먼저 죄송하단 말씀 드립니다 어제 저희 직원이 실수로 빨대를 넣지...,,1


## 긍부정 비율 확인
- 식당 별점 평가 사전학습 모델 : MLOpsEngineer/review-rate-prediction
- Label_0 = 1점, Label_1 = 2점, Label_2 = 3점, Label_3 = 4점, Label_4 = 5점


In [ ]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
print(train['review'][0])
classifier(train['review'][0])

사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱 맞춰서 보내주셨어요~~~~


[{'label': 'LABEL_3', 'score': 0.6671250462532043}]

In [ ]:
# 분류할 리뷰 텍스트
review_text = "맛있어요!"

# 모델 추론
result = classifier(review_text)[0]

# 결과 해석
label = result['label']
score = result['score']

if label in ['LABEL_0','LABEL_1'] :   # 어떤 기준으로 label 1, 2, 3을 잡을지 확인 필요. 대략은 0,1만 부정으로 하는게 좋아보임
    sentiment = '부정'
else:
    sentiment = '긍정'

# 출력
print(f"리뷰의 감정: {sentiment}")
print(f"긍부정 지수: {score}")


리뷰의 감정: 긍정
긍부정 지수: 0.5232095122337341


In [ ]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = '부정'
        else:
            sentiment = '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    df = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return df

In [ ]:
# train_posneg = analyze_reviews(train['review'])

In [ ]:
train_posneg

In [ ]:
# train_posneg.to_csv(f"{DATA_PATH}train_posneg.csv",index=False)
train_posneg = pd.read_csv(f"{DATA_PATH}train_posneg.csv")

In [ ]:
train_posneg

,Unnamed: 0,review,sentiment,score
0,0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,긍정,0.667125
1,1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,긍정,0.751605
2,2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,긍정,0.445775
3,3,굿 말이 필요 없는 메뉴 였습니다,긍정,0.741268
4,4,맛있어요 양도많구좋음,긍정,0.590259
...,...,...,...,...
18681,18681,맛있게 잘 먹었어요,긍정,0.491652
18682,18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,긍정,0.557971
18683,18683,배달 빨라요 맛있게 잘 먹었습니당 !,긍정,0.487945
18684,18684,잘 먹었어요~~~~~,긍정,0.512616


In [ ]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

,sentiment,score
0,긍정,0.667125
1,긍정,0.751605
2,긍정,0.445775
3,긍정,0.741268
4,긍정,0.590259
...,...,...
18681,긍정,0.491652
18682,긍정,0.557971
18683,긍정,0.487945
18684,긍정,0.512616


In [ ]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)
train.columns = ['review','reply','store','sentiment','score']

In [ ]:
# train.to_csv(f"{DATA_PATH}train_posneg_done.csv",index=False)

In [ ]:
train.head(10)

,review,reply,store,sentiment,score
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두...",,긍정,0.667125
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,,긍정,0.751605
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,,긍정,0.445775
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...,,긍정,0.741268
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...,,긍정,0.590259
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다 고객님들의 배고픔을 ...,,긍정,0.524225
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니...,정성 가득한 리뷰 작성해주셔서 감사합니다 맛있는 시간이 되셨다니 정말 다행입니다...,,긍정,0.765359
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,즐거운 순간을 저희와 함께해주셔서 감사합니다 믿고 찾아 주셔서 진심으로 감사드립...,,긍정,0.554175
8,맛있게 먹었습니다!!,귀한 발걸음 감사합니다!! 아낌없이 칭찬해 주셔서 기분이 좋습니다 저희 매장은...,,긍정,0.511925
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다...,귀한 발걸음 감사합니다!! 고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~~ ...,,긍정,0.732529


In [ ]:
train_posneg[mask].shape[0]   # 893개 : 부정 : 별점 비율과 유사함  (부정 별점 1~4 : 895개, 1~3 : 335개)

893

## 데이터 증강 (부정 리뷰)

In [ ]:
train= pd.read_csv(f"{DATA_PATH}train_posneg_done.csv")

In [ ]:
train['store'].isnull().sum()

12356

In [ ]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

893

In [ ]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

17793

## 1) resample 방법

In [ ]:
from sklearn.utils import resample

desired_samples = train[mask_pos].shape[0] - train[mask_neg].shape[0] #pos, neg개수 동일하게 하기 위해서
minority_class = train[train['sentiment'] == '부정']


In [ ]:
oversampled_data = resample(minority_class, replace=True, n_samples=desired_samples, random_state=42)
oversampled_data

,review,reply,store,sentiment,score
2049,죄송하지만 사장님.. 다시 안 시켜 먹을 것 같아요 쌀국수 양은 많은데 ...,녕하세요 고객님 정말정말 죄송합니다 부족한 부분으로 실망시켜 드린점 사...,배달의쌀국수,부정,0.942393
8972,육회는 맛있는데 전은 기름맛이 너무 심했어요. 육회는 정말 정말 맛있었습니다.,리뷰 감사합니다 전에 기름맛이 심하셨다니... 다음부터 더 신경쓰겠습니다 부...,NaN,부정,0.501978
17794,해장엔 커피 잘마셧어요,리뷰감사합니다! 저도 즐겨하는방법을....!! 반갑네요 즐거운 명절 되세요,NaN,부정,0.524454
5737,이게 제육이 맞나요.....??,"고객님, 인생제육을 주문해주셔서 진심으로 감사합니다. 열심히 하겠습니다! 그리고 인...",인생제육,부정,0.845268
2135,계속 벨 누르고 두드리고 문 앞에 두고 가시라니까 앞에 계시고 전화까지 하고 왜그...,안녕하세요 고객님 정말정말 죄송합니다 부족한 부분으로 실망시켜 드린점 ...,배달의쌀국수,부정,0.896142
...,...,...,...,...,...
4950,맛있었는데 5단계 시켰는데 전보다 꽤 덜 매웠어요 사람이 하는 일이니 오차가 있을수...,저팔계 중국 마라탕 덮밥 요리 입니다. 저희 음식을 맛있게 드시고 소중한 사...,저팔계,부정,0.420236
11739,두번째 주문인데 저번보단은 살이 퍽퍽해요 더끊여서 먹어야할듯 그리고 추가한 메추리알...,소중한 식사시간에 불편함드려 너무 죄송합니다 재료 하나하나 세밀하게 체크하고 있...,NaN,부정,0.898677
144,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다. 저희 1988응답하라추억의옛날도시락은 언제...,1988응답하라추억의옛날도시락,부정,0.837696
3776,냉면면이 특이해요 말먹었어요,안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다 항상 고객님들 주문 하나하나...,갈비민족,부정,0.406262


In [ ]:
aug_train = pd.concat([train,oversampled_data],axis=0)
aug_train.shape

(35586, 5)

In [ ]:
aug_train.to_csv(f"{DATA_PATH}yogiyo_aug_train")

## 2) EDA (easy data augmentation)

In [ ]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
# from nlpaug.util.text import random_word

# # EDA를 사용한 데이터 증강 예시 코드
# # 기존 리뷰를 다양한 방법으로 변형하여 새로운 리뷰 생성

# def eda_augmentation(review, alpha=0.1, num_aug=3):
#     augmented_reviews = []
#     words = review.split()

#     for _ in range(num_aug):
#         new_words = words.copy()
#         num_words_to_replace = max(1, int(alpha * len(words)))

#         for _ in range(num_words_to_replace):
#             rand_idx = random.randint(0, len(words) - 1)
#             new_words[rand_idx] = random_word(new_words[rand_idx])

#         augmented_reviews.append(' '.join(new_words))

#     return augmented_reviews

# # 부정 리뷰 데이터에 EDA 증강 적용
# final_oversampled_negative_reviews_eda = []
# for review in negative_reviews:
#     augmented_reviews = eda_augmentation(review)
#     final_oversampled_negative_reviews_eda.extend(augmented_reviews)


In [ ]:
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# import pandas as pd

# # 소수 클래스와 다수 클래스 데이터 분리
# minority_class = train[train['sentiment'] == '부정']
# majority_class = train[train['sentiment'] == '긍정']

# # SMOTE 적용
# smote = SMOTE(sampling_strategy='auto', random_state=42)
# oversampled_minority = smote.fit_resample(minority_class[['review','reply']], minority_class['sentiment'])

# # 부정 리뷰 데이터 증강된 데이터셋 생성
# augmented_data = pd.concat([majority_class, pd.DataFrame(oversampled_minority, columns=['review','reply', 'sentiment'])])

# # 증강 결과 확인
# print(Counter(augmented_data['sentiment']))


## 3) google 스토어리뷰 활용

In [ ]:
google = pd.read_csv(f"{DATA_PATH}playstore_df.csv")
google.head(10)

,content,score,replyContent
0,쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편...,2,"o Dan님,\n\n안녕하세요. 배달의민족입니다.\n혹시 이용에 불편함이나, 만족스..."
1,이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것,2,"문상화님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 이용 후 좋다고 말씀해주..."
2,이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요,1,"풍기도리님,\n\n안녕하세요. 배달의민족입니다.\n할인 혜택이 부족하여 아쉬움을 드..."
3,"아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...",1,"롤린이배린이 (롤배그올리는채널)님,\n\n안녕하세요. 배달의민족입니다.\n혜택 부족..."
4,좋아요!!,5,"꼬냑소년님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 서비스 이용에 만족하신..."
5,배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 ...,1,"김수하님,\n\n안녕하세요. 배달의민족입니다.\n이용 시 불편을 드린 것 같아 마음..."
6,응.,5,"bling Bin님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 소중한 리뷰 감..."
7,갑자기 어플이 안 들어가지는데 무슨 이유죠,3,"임종엽님,\n\n안녕하세요. 배달의민족입니다.\n앱 이용 시 불편을 드린 것 같아 ..."
8,"헐 배민1쿠폰은 쓸곳이 없는데 왜자꾸 준다고 난리난리이며, 8월 등급혜택쿠폰 쓰려다...",1,"최최님,\n\n안녕하세요. 배달의민족입니다.\n할인 혜택이 부족하여 아쉬움을 드린 ..."
9,처음설치해서쓰려는데 주소입력자체가안됨,1,"Jenny kim님,\n\n안녕하세요. 배달의민족입니다.\n앱 이용 시 불편을 드린..."


## 4) GAN (Generative Adversarial Networks)

# 데이터 학습

## 사전학습모델

In [ ]:
shuffle_aug_train = aug_train.sample(frac=1, random_state=42)
train_ft = shuffle_aug_train[:6000].astype(str)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

1

In [ ]:
tokenizer.tokenize(train['review'][0])

['▁사진',
 '보다',
 '▁실',
 '물이',
 '▁백',
 '배',
 '▁맛',
 '있어',
 '요',
 '~',
 '~',
 '~',
 '~',
 '~',
 '~',
 '▁고기',
 '▁양도',
 '▁많아',
 '요',
 '~',
 '~',
 '~',
 '~',
 '~',
 '~',
 '▁배달',
 '시간',
 '도',
 '▁딱',
 '▁맞춰',
 '서',
 '▁보내',
 '주',
 '셨',
 '어',
 '요',
 '~',
 '~',
 '~',
 '~']

## 데이터셋 만들기

In [ ]:
train_ft = train_ft[['review','reply','store']]

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.review = df["review"].tolist()
        self.reply = df["reply"].tolist()
        self.store = df["store"].tolist()

    def __len__(self):
        return len(self.review)

    def __getitem__(self,idx):
        return "<w>" + self.store[idx] + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"
                # 임의의 sos 토큰 <q>와 eos 토큰 </s> 삽입

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt", padding=True)
    return {"x":x}

In [ ]:
train_dt = ChatDataset(train_ft)

In [ ]:
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,collate_fn=collate_fn)

In [ ]:
batch = next(iter(train_dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,   452,  9574,     1,  9724,   455,   405,  8756,
           8346,  7380, 34398,  9054, 11355,  7055,  7098,  8084,   739,   739,
            739,     1,  9724,   439,   405, 17371,  7177, 16812,  8711,  9046,
           7810,  7788, 15940,  8708,  7172,  7182,   739, 16035,  9038,  9271,
           7187,  8704,  9234,  7445, 23939,  9228, 18651, 10133,  8185, 11714,
          15730,  9038, 29080,  7060,  9069,  7816,  9368,  9117,  7377,  7172,
           7182,   739,     1,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     

In [ ]:
tokenizer.pad_token_id # 패딩토큰인 3번 빼고 계산해야겠군

3

## 학습 loop함수화

In [ ]:
def train_loop(dataloader,model,optimizer,loss_fn,device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits # batch, seq, 단어별 실수값
        n_class = pred.shape[-1] # 클래스 개수
        pred = pred[:,:-1] # eos 토큰 제외
        pred = pred.reshape(-1,n_class) # 2차원 형태로 변환

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 변환

        mask = tgt != 3 # 패딩토큰인 3번 빼고 계산해야겠군
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

## 학습하기

In [ ]:
batch_size = 2
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
train_ft['review'] = train_ft['review'].astype(str)
train_ft['reply'] = train_ft['reply'].astype(str)
train_ft['store'] = train_ft['store'].astype(str)

In [ ]:
is_holdout = False
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, optimizer, loss_fn, device)
    print(train_loss)

    if is_holdout:
        break

  0%|          | 0/3000 [00:00<?, ?it/s]

1.9947832504014174


  0%|          | 0/3000 [00:00<?, ?it/s]

1.2438155172417562


  0%|          | 0/3000 [00:00<?, ?it/s]

...

0.34145118612299363


  0%|          | 0/3000 [00:00<?, ?it/s]

0.3017153906573852


  0%|          | 0/3000 [00:00<?, ?it/s]

0.2654216348665456


## 모델 저장하기

In [ ]:
model.save_pretrained(f"{DATA_PATH}yogiyo_kogpt2_model0830")

## 답변 생성

In [ ]:
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
@torch.no_grad()
def chatbot(model,tokenizer,max_len,device):
    model.eval()
    while True:
        text = input("소비자 > ").strip()
        if text == "quit":
            break
        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text,return_tensors="pt").to(device) # batch, seq

        result_ids = model.generate(x,
                            max_length=max_len,
                            repetition_penalty=2.0,
                            use_cache=True,
                            do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
                            temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
                            top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)
        q_len = len(text) + 1
        text = tokenizer.decode(result_ids[0])

        from hanspell import spell_checker

        text = correct_spell(text)

        # if len(text) > max_len:
        #     text = text[:max_len]

        print("사장님 > ",text[q_len:-4])

In [ ]:

chatbot(model,tokenizer,128,device) # quit

# 답변 평가

## PPL

In [ ]:
import torch
import torch.nn as nn

# # 가정: 언어 모델 클래스가 이미 정의되어 있다고 가정
# class LanguageModel(nn.Module):
#     # 모델 정의 ...

# 퍼플렉시티 계산 함수
def calculate_perplexity(model, dataloader, loss_function):
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()
            total_tokens += targets.numel()

    avg_loss = total_loss / total_tokens
    perplexity = 2 ** avg_loss  # 퍼플렉시티 계산

    return perplexity

# 사용 예시
# model = LanguageModel(...)  # 언어 모델 클래스 초기화
# dataloader = torch.utils.data.DataLoader(...)  # 데이터로더 초기화
# loss_function = nn.CrossEntropyLoss()  # 손실 함수 초기화
# perplexity = calculate_perplexity(model, dataloader, loss_function)
# print(f"Perplexity: {perplexity}")


## BLEU

## METEOR

## SSA